In [ ]:
import tensorflow as tf
import keras

from keras import models, layers, Input
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt



In [ ]:
def plot_loss(history):
  """ Plot training, test loss. """
  
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('model loss')
  plt.ylabel('loss')
  plt.xlabel('epoch')
  plt.legend(['train', 'test'])
  plt.show();

## Logistic Regression


In [ ]:
model = models.Sequential()
model.add(Input(shape=(4,)))
model.add(layers.Dense(1, activation='sigmoid'))
model.summary()


In [ ]:
model.compile(optimizer='SGD',
  loss='binary_crossentropy',
  metrics=['accuracy']
)

In [ ]:
from sklearn.datasets import make_classification

X,y = make_classification(
  n_samples=1000, 
  n_features=4, 
  n_redundant=0
)

X_train, X_test, y_train, y_test = train_test_split(
  X, 
  y, 
  test_size=0.33, 
  random_state=42
)


In [ ]:
history = model.fit(X_train, y_train,
  batch_size=32, 
  epochs=400, 
  validation_split=0.3
)


In [ ]:
print(history.history.keys())


In [ ]:
plot_loss(history)

## Derived Features

In [ ]:
# one hidden layer of three neurons, sigmoids throughout
model_sigmoid = models.Sequential()
model_sigmoid.add(layers.InputLayer(shape=(4,)))
model_sigmoid.add(layers.Dense(3, activation='sigmoid'))
model_sigmoid.add(layers.Dense(1, activation='sigmoid'))
model_sigmoid.summary()

model_sigmoid.compile(optimizer='SGD',
  loss='binary_crossentropy',
  metrics=['accuracy']
)

history_sigmoid = model_sigmoid.fit(
  X_train, 
  y_train,
  batch_size=512, 
  epochs=1000, 
  validation_split=0.3
)
plot_loss(history_sigmoid)


In [ ]:
# one hidden layer of three neurons, relu in the middle
model_relu = models.Sequential()
model_relu.add(layers.InputLayer(input_shape=(4,)))
model_relu.add(layers.Dense(3, activation='relu'))
model_relu.add(layers.Dense(1, activation='sigmoid'))
model_relu.summary()

model_relu.compile(optimizer='SGD',
  loss='binary_crossentropy',
  metrics=['accuracy']
)

history_relu = model_relu.fit(
  X_train, 
  y_train,
  batch_size=256, 
  epochs=40, 
  validation_split=0.3
)
plot_loss(history_relu)

In [ ]:
# one hidden layer of three neurons, relu in the middle
model_relu = models.Sequential()
model_relu.add(layers.InputLayer(input_shape=(4,)))
model_relu.add(layers.Dense(3, activation='relu'))
model_relu.add(layers.Dense(1, activation='relu'))
model_relu.add(layers.Softmax(1))
model_relu.summary()

model_relu.compile(optimizer='SGD',
  loss='binary_crossentropy',
  metrics=['accuracy']
)

history_relu = model_relu.fit(
  X_train,
  y_train,
  batch_size=256,
  epochs=40,
  validation_split=0.3
)
plot_loss(history_relu)

## Richer Models


In [ ]:
# two hidden layers, relu activation
model_rich = models.Sequential()
model_rich.add(layers.InputLayer(input_shape=(4,)))
model_rich.add(layers.Dense(20, activation='relu'))
model_rich.add(layers.Dense(20, activation='relu'))
model_rich.add(layers.Dense(1, activation='sigmoid'))


model_rich.summary()


In [ ]:

model_rich.compile(optimizer='SGD',
  loss='binary_crossentropy',
  metrics=['accuracy']
)

history_rich = model_rich.fit(
  X_train,
  y_train,
  batch_size=256,
  epochs=1000,
  validation_split=0.3
)
plot_loss(history_rich)

## Multi-class


In [ ]:
X,y = make_classification(
  n_samples=1000,
  n_features=4, 
  n_redundant=0, 
  n_classes=3,
  n_clusters_per_class=1
)

X_train, X_test, y_train, y_test = train_test_split(
  X, 
  y, 
  test_size=0.33, 
  random_state=42
)


In [ ]:
# three output classes; softmax is used
model_mc = models.Sequential()
model_mc.add(layers.InputLayer(input_shape=(4,)))
model_mc.add(layers.Dense(20, activation='relu'))
model_mc.add(layers.Dense(20, activation='relu'))
model_mc.add(layers.Dense(3, activation='softmax'))

model_mc.summary()


In [ ]:

model_mc.compile(
  optimizer='SGD',
  loss='sparse_categorical_crossentropy',
  metrics=['accuracy']
)

history_mc = model_mc.fit(
  X_train, 
  y_train,
  batch_size=256, 
  epochs=1000, 
  validation_split=0.3
)

plot_loss(history_mc)


In [ ]:
# three output classes; softmax is used
model_mc1 = models.Sequential()
model_mc1.add(layers.InputLayer(input_shape=(4,)))
model_mc1.add(layers.Dense(20, activation='relu'))
model_mc1.add(layers.Dense(20, activation='relu'))
model_mc1.add(layers.Dense(3, activation='softmax'))

model_mc1.summary()

model_mc1.compile(
  optimizer='SGD',
  loss='sparse_categorical_crossentropy',
  metrics=['accuracy']
)

history_mc = model_mc1.fit(
  X_train,
  y_train,
  batch_size=512,
  epochs=1000,
  validation_split=0.3
)

plot_loss(history_mc)
